In [6]:
import os
os.chdir("../")
%pwd

'/Users/valentinmonney/Documents/data_science/python/MachineLearningProjects/Kidney-Disease-Classification-Deep-Learning-Project'

In [7]:
os.environ[
    "MLFLOW_TRACKING_URI"
] = "https://dagshub.com/vmonney/Kidney-Disease-Classification-Deep-Learning-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "vmonney"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "6606689713d41518007b9029977da220960be7ce"

In [8]:
import tensorflow as tf

In [9]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    """Configuration class for evaluation."""

    path_of_model: Path
    training_data: Path
    mlflow_uri: str
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [13]:
from cnn_classifier.constants import *

In [16]:
class ConfigurationManager:
    """Class for managing configuration settings."""

    def __init__(
        self,
        config_filepath: str = CONFIG_FILE_PATH,
        params_filepath: str = PARAMS_FILE_PATH,
    ) -> None:
        """Initialize the ConfigurationManager class.

        Args:
        ----
            config_filepath: The path to the configuration file.
            params_filepath: The path to the parameters file.
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        """Return the validation configuration."""
        return EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
            mlflow_uri="https://dagshub.com/vmonney/Kidney-Disease-Classification-Deep-Learning-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
        )

In [17]:
class Evaluation:
    """Class for evaluating a trained model."""

    def __init__(self, config: EvaluationConfig) -> None:
        """Initialize the Evaluation class.

        Args:
        ----
          config (EvaluationConfig): An instance of EvaluationConfig
          containing the configuration for evaluation.
        """
        self.config = config

    def _valid_generator(self) -> None:
        datagenerator_kwargs = {"rescale": 1.0 / 255, "validation_split": 0.30}

        dataflow_kwargs = {
            "target_size": self.config.params_image_size[:-1],
            "batch_size": self.config.params_batch_size,
            "interpolation": "bilinear",
        }

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs,
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs,
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        """Load model."""
        return tf.keras.models.load_model(path)

    def evaluation(self) -> None:
        """Evaluate the trained model."""
        model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self) -> None:
        """Save the evaluation score to a JSON file."""
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(self.config.mlflow_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric({"loss": self.score[0], "accuracy": self.score[1]})
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(
                    self.model, "model", registered_model_name="VGG16Model"
                )
            else:
                mlflow.keras.log_model(model, "model")

In [ ]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise